In [ ]:
import yaml
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt

# Uncomment if SimBMVtool not in directory
# import sys
# path_SimBMVtool = '<path_to_SimBMVtool>'
# sys.path.append(path_SimBMVtool)

from SimBMVtool.sim_bmv_creator import SimBMVCreator
from SimBMVtool.toolbox import (
    get_skymaps_dict,
    plot_skymap_from_dict
)
path_config = './config_simu.yaml'

In [ ]:
simbmv = SimBMVCreator(path_config)
simbmv.load_true_background_irfs() 
simbmv.load_output_background_irfs()

if simbmv.out_collection:
    simbmv.bkg_output_irf_collection[1].plot_at_energy(["1 TeV"],figsize=(3,3))
    simbmv.bkg_output_irf_collection[1].peek()
else: 
    simbmv.bkg_output_irf.plot_at_energy(["1 TeV"],figsize=(3,3))
    simbmv.bkg_output_irf.peek()

In [ ]:
plot_data = True
if plot_data:
    # TO-DO: re-implement zenith binned plotting within plot_method
    icos='all'
    simbmv.plot_model(data='acceptance', irf='true', residuals='none',title="Acceptance: true", fig_save_path=f"{simbmv.plots_dir}/acceptance_true_cosbin_{icos}.png")
    simbmv.plot_model(data='acceptance', irf='output', residuals='none',title="Acceptance: output", fig_save_path=f"{simbmv.plots_dir}/acceptance_out_cosbin_{icos}.png")
    simbmv.plot_model(data='acceptance', irf='both', residuals='diff/true',title="Acceptance: residuals", fig_save_path=f"{simbmv.plots_dir}/acceptance_residuals_cosbin_{icos}.png", plot_hist=True)
    simbmv.plot_model(data='acceptance', irf='both', residuals='diff/sqrt(true)', title="Acceptance: residuals", fig_save_path=f"{simbmv.plots_dir}/acceptance_residuals_sqrt_cosbin_{icos}.png")

In [ ]:
plot_profiles = True

if plot_profiles:
    simbmv.plot_profile(irf='output', i_irf=1, profile='both', stat='sum', bias=False, ratio_lim=[0.95,1.05], all_Ebins=True, fig_save_path=f"{simbmv.plots_dir}/profile_stat_sum.png")
    simbmv.plot_profile(irf='both', i_irf=1, profile='both', stat='mean', bias=True, ratio_lim=[0.95,1.05], all_Ebins=True, fig_save_path=f"{simbmv.plots_dir}/profile_bias_mean.png")
    simbmv.plot_profile(irf='both', i_irf=1, profile='both', stat='std', bias=True, ratio_lim=[0.95,1.05], all_Ebins=True, fig_save_path=f"{simbmv.plots_dir}/profile_bias_std.png")

In [ ]:
# Check what your output skymaps should look like
plot_skymaps  = True
if plot_skymaps:
    simbmv.load_observation_collection()  # Load simulated data with true background model
    simbmv.axis_info_dataset = [simbmv.e_min, simbmv.e_max, simbmv.size_fov_acc, 10 * simbmv.nbin_offset_acc]
    simbmv.plot_skymaps('ring')           # Ring background method is applied then all energies are stacked
    simbmv.plot_skymaps('FoV')

In [ ]:
# Load simulated data with output model, uses index tables based on naming scheme following config file info
simbmv.load_observation_collection(from_index=True)               

# Plot skymaps for each energy bin
for (e_min_map,e_max_map) in simbmv.Ebin_tuples:
    simbmv.axis_info_dataset = [e_min_map * u.TeV, e_max_map * u.TeV, simbmv.size_fov_acc, 10 * simbmv.nbin_offset_acc]
    simbmv.plot_skymaps('ring')
    plt.show()

# You see that the maps are not very good for larger energy bins with less statistics
# The model residuals show empty bins at large offset, with large mean bias above 4°
# So we need to plot each energy bins with offset safe mask depending on the output model
# It is better, but to get results similar to the previous cell, you would need more simulated livetime to have less empty bins
# or to use the fit method to perform the background model

for (e_min_map,e_max_map), offset_max_dataset in zip(simbmv.Ebin_tuples, [4., 3., 2.]):
    simbmv.axis_info_dataset = [e_min_map * u.TeV, e_max_map * u.TeV, offset_max_dataset * u.deg, 10 * simbmv.nbin_offset_acc]
    simbmv.plot_skymaps('ring')
    plt.show()

In [ ]:
e_min_map, e_max_map = (0.5 * u.TeV, 10. * u.TeV)
offset_max_dataset = 1.9 * u.deg                          # Using the safest offset max to stack all energies

# If you want to change the exclusion region radius, you can change the parameters directly and update the exclusion region
# Here is how to change the n_circles parameters
# In this case we simulated without any source so the results should be almost identical.
# As an exercice, you can try to simulate with a source and look at the difference

i_circle = 0 
exclusion_radius_arr = [0.2, 0.25, 0.3]

for i_radbin, radius in enumerate(exclusion_radius_arr):
    simbmv.n_circles_radius[0] = radius
    simbmv.region_shape = "n_circles" if radius > 0 else "noexclusion"
    simbmv.init_exclusion_region(cfg_exclusion_region='stored_parameters', init_save_paths=True)

    simbmv.do_baccmod()
    
    simbmv.axis_info_dataset = [e_min_map, e_max_map, offset_max_dataset, 8*simbmv.nbin_offset_acc]
    simbmv.plot_skymaps('ring')
    plt.show()

In [ ]:
# Plotting everything takes some time, and you may need to look only at one map
stacked_dataset = simbmv.get_stacked_dataset(bkg_method='ring', axis_info=simbmv.axis_info_dataset)

skymaps = get_skymaps_dict(stacked_dataset, simbmv.exclude_regions, simbmv.correlation_radius, simbmv.correlate_off, 'all')
print(f"maps in dict: {[key for key in skymaps.keys()]}")

plot_skymap_from_dict(skymaps, 'counts', figsize=(4,4))

# Plot just the off significance map with ring background kernel for informative purpose, and crop the map
plot_skymap_from_dict(skymaps, 'significance_off', ring_bkg_param=simbmv.ring_bkg_param, crop_width=1.5*u.deg)

In [ ]:
# Now with the fit method there are no empty bins, the skymaps can be plotted up to 5°
simbmv.method = 'fit'
simbmv.init_save_paths()

# Uncomment if you haven't already done the modeling
# simbmv.do_baccmod()
 
simbmv.load_output_background_irfs()
simbmv.load_observation_collection(from_index=True) # Uses index tables based on automatic naming scheme

plot_skymaps  = True
if plot_skymaps:
    simbmv.axis_info_dataset = [simbmv.e_min, simbmv.e_max, simbmv.size_fov_acc, 10 * simbmv.nbin_offset_acc]
    simbmv.axis_info_map = [simbmv.e_min, simbmv.e_max, simbmv.size_fov_acc, 10 * simbmv.nbin_offset_acc]
    simbmv.plot_skymaps('ring')
    simbmv.plot_skymaps('FoV')